In [1]:
import os
import librosa
import numpy as np
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Input
from tensorflow.keras.applications import EfficientNetB0
from tensorflow.keras.optimizers import Adam
from sklearn.metrics import classification_report

In [2]:
# Hàm trích xuất MFCC
def extract_mfcc(audio, sr=16000, n_mfcc=40):
    hop_length = int(0.01 * sr)  # 10 ms hop
    win_length = int(0.025 * sr)  # 25 ms window
    mfcc = librosa.feature.mfcc(y=audio, sr=sr, n_mfcc=n_mfcc, hop_length=hop_length, win_length=win_length)
    return np.expand_dims(mfcc, axis=-1)  # Thêm chiều kênh (cho CNN)

# Hàm tải dữ liệu
def load_data(data_dir):
    X = []
    y = []
    labels = os.listdir(data_dir)
    for label in labels:
        label_dir = os.path.join(data_dir, label)
        for file_name in os.listdir(label_dir):
            if file_name.endswith(".wav"):
                file_path = os.path.join(label_dir, file_name)
                # Tải tệp âm thanh
                audio, sr = librosa.load(file_path, sr=None)
                features = extract_mfcc(audio, sr)
                X.append(features)
                y.append(label)
    return np.array(X), np.array(y)


In [3]:
# Load dữ liệu
train_data_dir = r'C:\Users\USER\Downloads\SV_NCKH_audio_event\Train'
test_data_dir = r'C:\Users\USER\Downloads\SV_NCKH_audio_event\Test'

X_train, y_train = load_data(train_data_dir)
X_test, y_test = load_data(test_data_dir)

# Encode nhãn
le = LabelEncoder()
y_train = le.fit_transform(y_train)
y_test = le.transform(y_test)

# One-hot encode nhãn
num_classes = len(np.unique(y_train))
y_train = to_categorical(y_train, num_classes=num_classes)
y_test = to_categorical(y_test, num_classes=num_classes)

# Định dạng đầu vào
X_train = X_train.astype("float32")
X_test = X_test.astype("float32")

In [4]:
# EfficientNet model
def build_efficientnet(input_shape, num_classes):
    # Sử dụng EfficientNetB0 làm base model
    base_model = EfficientNetB0(include_top=False, weights=None, input_shape=input_shape)
    
    # Thêm các lớp phía trên
    x = GlobalAveragePooling2D()(base_model.output)
    outputs = Dense(num_classes, activation="softmax")(x)
    
    model = Model(inputs=base_model.input, outputs=outputs)
    model.compile(optimizer=Adam(learning_rate=0.001), loss="categorical_crossentropy", metrics=["accuracy"])
    return model

# Xây dựng EfficientNet
input_shape = X_train.shape[1:]  # Dữ liệu đầu vào có shape (tần số, thời gian, kênh)
model = build_efficientnet(input_shape, num_classes)
model.summary()

# Huấn luyện mô hình
model.fit(X_train, y_train, batch_size=32, epochs=50, validation_data=(X_test, y_test))

Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer         │ (None, 40, 101,   │          0 │ -                 │
│ (InputLayer)        │ 1)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ rescaling           │ (None, 40, 101,   │          0 │ input_layer[0][0] │
│ (Rescaling)         │ 1)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ normalization       │ (None, 40, 101,   │          3 │ rescaling[0][0]   │
│ (Normalization)     │ 1)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ stem_conv_pad       │ (None, 41, 103,   │          0 │ normalization[0]… │
│ (ZeroPadding2D)     │ 1)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ stem_conv (Conv2D)  │ (None, 20, 51,    │        288 │ stem_conv_pad[0]… │
│                     │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ stem_bn             │ (None, 20, 51,    │        128 │ stem_conv[0][0]   │
│ (BatchNormalizatio… │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ stem_activation     │ (None, 20, 51,    │          0 │ stem_bn[0][0]     │
│ (Activation)        │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1a_dwconv      │ (None, 20, 51,    │        288 │ stem_activation[… │
│ (DepthwiseConv2D)   │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1a_bn          │ (None, 20, 51,    │        128 │ block1a_dwconv[0… │
│ (BatchNormalizatio… │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1a_activation  │ (None, 20, 51,    │          0 │ block1a_bn[0][0]  │
│ (Activation)        │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1a_se_squeeze  │ (None, 32)        │          0 │ block1a_activati… │
│ (GlobalAveragePool… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1a_se_reshape  │ (None, 1, 1, 32)  │          0 │ block1a_se_squee… │
│ (Reshape)           │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1a_se_reduce   │ (None, 1, 1, 8)   │        264 │ block1a_se_resha… │
│ (Conv2D)            │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1a_se_expand   │ (None, 1, 1, 32)  │        288 │ block1a_se_reduc… │
│ (Conv2D)            │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1a_se_excite   │ (None, 20, 51,    │          0 │ block1a_activati… │
│ (Multiply)          │ 32)               │            │ block1a_se_expan… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1a_project_co… │ (None, 20, 51,    │        512 │ block1a_se_excit… │
│ (Conv2D)            │ 16)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1a_project_bn  │ (None, 20, 51,    │         64 │ block1a_project_

 Total params: 4,075,892 (15.55 MB)

 Trainable params: 4,033,873 (15.39 MB)

 Non-trainable params: 42,019 (164.14 KB)

Epoch 1/50
74/74 ━━━━━━━━━━━━━━━━━━━━ 148s 1s/step - accuracy: 0.2058 - loss: 2.6987 - val_accuracy: 0.1207 - val_loss: 2.9059
Epoch 2/50
74/74 ━━━━━━━━━━━━━━━━━━━━ 66s 837ms/step - accuracy: 0.4747 - loss: 1.7244 - val_accuracy: 0.0878 - val_loss: 3.1445
Epoch 3/50
74/74 ━━━━━━━━━━━━━━━━━━━━ 63s 856ms/step - accuracy: 0.5804 - loss: 1.2915 - val_accuracy: 0.1427 - val_loss: 3.1883
Epoch 4/50
74/74 ━━━━━━━━━━━━━━━━━━━━ 102s 1s/step - accuracy: 0.6820 - loss: 1.0660 - val_accuracy: 0.0247 - val_loss: 4.6785
Epoch 5/50
74/74 ━━━━━━━━━━━━━━━━━━━━ 90s 1s/step - accuracy: 0.7437 - loss: 0.8036 - val_accuracy: 0.0864 - val_loss: 6.0068
Epoch 6/50
74/74 ━━━━━━━━━━━━━━━━━━━━ 74s 994ms/step - accuracy: 0.7787 - loss: 0.6787 - val_accuracy: 0.1344 - val_loss: 5.7764
Epoch 7/50
74/74 ━━━━━━━━━━━━━━━━━━━━ 90s 1s/step - accuracy: 0.8466 - loss: 0.4903 - val_accuracy: 0.1578 - val_loss: 7.0345
Epoch 8/50
74/74 ━━━━━━━━━━━━━━━━━━━━ 79s 1s/step - accuracy: 0.8574 - loss: 0.4728 - val_accuracy: 0.4005 